In [1]:
import numpy as np
import math

# C. 天空日射・反射日射の効果係数 (仕様書6.4, 6.5)

## C.1 形態係数算定のための関数$\;f_A\;$ (仕様書6.4 式(23))


$$ \begin{align}
f_{A}(x_a,x_b,y_a,y_b,z_a) &= \frac{x_b \sqrt{y_b^2+z_a^2}}{2} \tan^{-1} \frac{x_b}{\sqrt{y_b^2+z_a^2}}
- \frac{x_b \sqrt{y_a^2+z_a^2}}{2} \tan^{-1} \frac{x_b}{\sqrt{y_a^2+z_a^2}} \\
&- \frac{x_a \sqrt{y_b^2+z_a^2}}{2} \tan^{-1} \frac{x_a}{\sqrt{y_b^2+z_a^2}}
+ \frac{x_a \sqrt{y_a^2+z_a^2}}{2} \tan^{-1} \frac{x_a}{\sqrt{y_a^2+z_a^2}} \\
&+ \frac{x_b^2 - y_b^2 - z_a^2}{8} \log (x_b^2 + y_b^2 + z_a^2) - \frac{x_b^2 - y_a^2 - z_a^2}{8} \log (x_b^2 + y_a^2 + z_a^2) \\
&- \frac{x_a^2 - y_b^2 - z_a^2}{8} \log (x_a^2 + y_b^2 + z_a^2) + \frac{x_a^2 - y_a^2 - z_a^2}{8} \log (x_a^2 + y_a^2 + z_a^2)
\end{align} $$

In [2]:
def calc_fa_atan(x, y, z):

    if y**2 + z**2 > 0:
        return x * ( y**2 + z**2 ) **0.5 / 2 * math.atan( x / ( y**2 + z**2 ) **0.5 )
    else:
        return 0.0

def calc_fa_log(x, y, z):

    if x**2 + y**2 + z**2 > 0:
        return ( x**2 - y**2 - z**2 ) / 8 * math.log( x**2 + y**2 + z**2 )
    else:
        return 0.0

def calc_fa(xa, xb, ya, yb, za):
    
    return ( calc_fa_atan(xb, yb, za) - calc_fa_atan(xb, ya, za)
           - calc_fa_atan(xa, yb, za) + calc_fa_atan(xa, ya, za)
           + calc_fa_log(xb, yb, za)  - calc_fa_log(xb, ya, za)
           - calc_fa_log(xa, yb, za)  + calc_fa_log(xa, ya, za) )

## C.2 天空に対する形態係数 (仕様書6.4 式(22))

$$ \begin{align}
\phi_{j,y+} = (\pi A_{wind,j})^{-1} \{ & \hspace{2pt} f_A(x_{3y+}, x_2 + x_{3y+}, y_1, y_1 + y_2, z_{y+}) + f_A(y_{1x+}, y_{1x+} + y_2, x_{3}, x_2 + x_{3}, z_{x+}) \\
+ & \hspace{2pt} f_A(x_{1y+}, x_{1y+} + x_2, y_1, y_1 + y_2, z_{y+}) + f_A(y_{1x-}, y_{1x-} + y_2, x_{1}, x_1 + x_2, z_{x-}) \\ 
+ & \hspace{2pt} f_A(x_{3}, x_2 + x_{3}, y_1, y_1 + y_2, 0) + f_A(y_{1}, y_{1} + y_2, x_{3}, x_2 + x_{3}, 0) \\
+ & \hspace{2pt} f_A(x_{1}, x_{1} + x_2, y_1, y_1 + y_2, 0) + f_A(y_{1}, y_{1} + y_2, x_{1}, x_1 + x_2, 0) \\ 
- & \hspace{2pt} f_A(x_{3y+}, x_2 + x_{3y+}, y_1, y_1 + y_2, 0) - f_A(y_{1x+}, y_{1x+} + y_2, x_{3}, x_2 + x_{3}, 0) \\
- & \hspace{2pt} f_A(x_{1y+}, x_{1y+} + x_2, y_1, y_1 + y_2, 0) - f_A(y_{1x-}, y_{1x-} + y_2, x_{1}, x_1 + x_2, 0) \\
\end{align} $$

In [3]:
def calc_phiyp(WSSize):
    
    X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym = WSSize
    
    phiyp = ( 1 / ( np.pi * X2 * Y2 )
            * ( calc_fa(X3yp, X2 + X3yp, Y1, Y1 + Y2, Zyp) + calc_fa(Y1xp, Y1xp + Y2, X3, X2 + X3, Zxp) 
              + calc_fa(X1yp, X1yp + X2, Y1, Y1 + Y2, Zyp) + calc_fa(Y1xm, Y1xm + Y2, X1, X1 + X2, Zxm)  
              + calc_fa(X3,   X2  +  X3, Y1, Y1 + Y2, 0  ) + calc_fa(Y1,   Y1  +  Y2, X3, X2 + X3, 0  ) 
              + calc_fa(X1,   X1  +  X2, Y1, Y1 + Y2, 0  ) + calc_fa(Y1,   Y1  +  Y2, X1, X1 + X2, 0  )     
              - calc_fa(X3yp, X2 + X3yp, Y1, Y1 + Y2, 0  ) - calc_fa(Y1xp, Y1xp + Y2, X3, X2 + X3, 0  ) 
              - calc_fa(X1yp, X1yp + X2, Y1, Y1 + Y2, 0  ) - calc_fa(Y1xm, Y1xm + Y2, X1, X1 + X2, 0  ) ) ) 
    return max(0, min(phiyp, 0.5))    #負値は0に、0.5を超える場合は0.5で頭打ち

## C.3 天空日射の効果係数 (仕様書6.4 式(21))

$$\gamma_{isr,j,y+} = 2 \phi_{j,y+} $$

In [4]:
def calc_gammayp(WSSize):
    
    return 2 * calc_phiyp(WSSize)

## C.4 地面に対する形態係数 (仕様書6.5 式(25))

$$ \begin{align}
\phi_{j,y-} = (\pi A_{wind,j})^{-1} \{ & \hspace{2pt} f_A(x_{3y-}, x_2 + x_{3y-}, y_3, y_2 + y_3, z_{y-}) + f_A(y_{3x+}, y_2 + y_{3x+}, x_{3}, x_2 + x_{3}, z_{x+}) \\
+ & \hspace{2pt} f_A(x_{1y-}, x_{1y-} + x_2, y_3, y_2 + y_3, z_{y-}) + f_A(y_{3x-}, y_2 + y_{3x-}, x_{1}, x_1 + x_2, z_{x-}) \\ 
+ & \hspace{2pt} f_A(x_{3}, x_2 + x_{3}, y_3, y_2 + y_3, 0) + f_A(y_{3}, y_{2} + y_3, x_{3}, x_2 + x_{3}, 0) \\
+ & \hspace{2pt} f_A(x_{1}, x_{1} + x_2, y_3, y_2 + y_3, 0) + f_A(y_{3}, y_{2} + y_3, x_{1}, x_1 + x_2, 0) \\ 
- & \hspace{2pt} f_A(x_{3y-}, x_2 + x_{3y-}, y_3, y_2 + y_3, 0) - f_A(y_{3x+}, y_2 + y_{3x+}, x_{3}, x_2 + x_{3}, 0) \\
- & \hspace{2pt} f_A(x_{1y-}, x_{1y-} + x_2, y_3, y_2 + y_3, 0) - f_A(y_{3x-}, y_2 + y_{3x-}, x_{1}, x_1 + x_2, 0) \\
\end{align} $$

In [5]:
def calc_phiym(WSSize):
    
    X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym = WSSize
    
    phiym = ( 1 / ( np.pi * X2 * Y2 )
            * ( calc_fa(X3ym, X2 + X3ym, Y3, Y2 + Y3, Zym) + calc_fa(Y3xp, Y2 + Y3xp, X3, X2 + X3, Zxp) 
              + calc_fa(X1ym, X1ym + X2, Y3, Y2 + Y3, Zym) + calc_fa(Y3xm, Y2 + Y3xm, X1, X1 + X2, Zxm)  
              + calc_fa(X3,   X2  +  X3, Y3, Y2 + Y3, 0  ) + calc_fa(Y3,   Y2  +  Y3, X3, X2 + X3, 0  ) 
              + calc_fa(X1,   X1  +  X2, Y3, Y2 + Y3, 0  ) + calc_fa(Y3,   Y2  +  Y3, X1, X1 + X2, 0  )     
              - calc_fa(X3ym, X2 + X3ym, Y3, Y2 + Y3, 0  ) - calc_fa(Y3xp, Y2 + Y3xp, X3, X2 + X3, 0  ) 
              - calc_fa(X1ym, X1ym + X2, Y3, Y2 + Y3, 0  ) - calc_fa(Y3xm, Y2 + Y3xm, X1, X1 + X2, 0  ) ) ) 
    return max(0, min(phiym, 0.5))    #負値は0に、0.5を超える場合は0.5で頭打ち


## C.5 反射日射の効果係数 (仕様書6.5 式(24))

$$\gamma_{isr,j,y-} = 2 \phi_{j,y-} $$

In [6]:
def calc_gammaym(WSSize):

    return 2 * calc_phiym(WSSize)